# 1.1 Create PyBossa Tasks
Create tasks in Pybossa using a Geojson file containing polyongs (e.g. buildings).

In [8]:
from IPython.display import *
from IPython.display import HTML
import json
import numpy as np
import pandas as pd
import pbclient

import json
import geojson
from shapely.geometry import shape
from utils_pybossa_functions import *

## Configuration
Information on project_id etc. 

In [9]:
sample_data_file = './data/2989_buildings_oriented_bbox_select_aligned_random.geojson'

config_file = './cfg/config.json'
with open(config_file) as f:
    config = json.load(f)

project_shortname = 'mapswipe_2_0'
project_id = 49

# setup the pybossa server connection
pbclient.set('endpoint', config['pybossa']['endpoint'])
pbclient.set('api_key', config['pybossa']['key'])

pybossa_project = pbclient.get_project(project_id)
print(pybossa_project)

pybossa.Project("mapswipe_2_0", 49)


## Create Task in PyBossa


In [11]:
with open(sample_data_file) as f:
    geojson_data = geojson.load(f)

all_task_ids =  []

all_tasks = pbclient.get_tasks(pybossa_project.id, last_id=None)
    
# let's get the id of the tasks we already imported
# we can only request 100 tasks at a time
while len(all_tasks) > 0:
    # delete tasks one after the other
    for task in all_tasks:
        all_task_ids.append(task.info['id'])
        last_id = task.id

    all_tasks = pbclient.get_tasks(pybossa_project.id, last_id=last_id)

duplicates = 0
created = 0
for g in geojson_data['features']:
    object_id = g['properties']['id']
    
    if object_id in all_task_ids:
        duplicates += 1
        continue
    
    geometry = g['geometry']
    wkt_geometry = shape(geometry).wkt
    
    task_info = {
        'id': object_id,
        'wkt_geom': wkt_geometry
    }
    
    response = pbclient.create_task(pybossa_project.id, task_info)
    created += 1
    
print('created tasks: %s, not created because duplicate: %s' % (created, duplicates))


    INFO: you can use keyset pagination to get faster responses from the server.
    To learn more, please visit:
    https://github.com/PYBOSSA/pybossa-client#on-queries-and-performance
    
created tasks: 3, not created because duplicate: 997


## Delete Tasks and Task Runs in Pybossa

In [7]:
pybossa_project = pbclient.find_project(short_name=project_shortname)[0]
#delete_all_pybossa_tasks(pbclient, pybossa_project)
delete_all_pybossa_taskruns(pbclient, pybossa_project)


    INFO: you can use keyset pagination to get faster responses from the server.
    To learn more, please visit:
    https://github.com/PYBOSSA/pybossa-client#on-queries-and-performance
    
after deleting 20 taskruns there are 0 taskruns for project: pybossa.Project("mapswipe_2_0", 49)
